### DESCRIPTION

Run S2A systems technoeconomic analysis. Run "s2a_systems_functions.py" for user-specified input parameters.

Outputs:
- "input params.csv": all sets of input parameters mapped to run #.
- "input params_varied.csv": input parameters that are different from baseline input parameters (run 0) mapped to run #.
- "output_[xxxx].csv": input parameters and results for each run. Filename suffix = run #.
- "output_all runs.csv": input parameters and results for all runs.
- "output_all runs_refueling station costs.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs.
- "output_all runs_refueling station costs_pivot.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs, pivot format.
- "output_cost by [pathway/process/location/function/process-function].csv": input parameters and cost results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.
- "output_emissions by [pathway/process/location/function/process-function].csv": input parameters and emissions results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.

### IMPORT MODULES

In [1]:
import s2a_systems_functions as s2a_sys
import numpy as np
import pandas as pd
import os
from datetime import date

In [2]:
# show default arguments for reference
help(s2a_sys.calcs)

Help on function calcs in module s2a_systems_functions:

calcs(dict_input_params={'run #': 0, 'output dollar year': 2022, 'target station capacity (kg/day)': 1000.0, 'station annual capacity factor': 1.0, 'target number of stations': 10, 'one-way delivery distance (mile)': 100.0, 'electricity cost ($/kWh)': 0.1709, 'diesel cost ($/gallon)': 6.028, 'electricity emission factor (kg CO2/kWh)': 0.228, 'diesel emission factor (kg CO2/gallon)': 10.18, 'hydrogen prod. emission factor (kg CO2-eq/kg)': 0.0, 'LOHC prod. emission factor (kg CO2-eq/kg)': 0.0, 'hydrogen purchase cost ($/kg)': 0.31, 'LOHC purchase cost ($/kg)': 1.0, 'LOHC name': 'formic acid', 'LOHC molar mass (kg/kmol)': 46.025, 'LOHC density (kg/m^3)': 1220.0, 'stoic. ratio (mol H2/mol LOHC)': 1, 'stoic. ratio (mol CO2/mol LOHC)': 1, 'hydr. reaction pathway': 'electro', 'hydr. reaction temperature (K)': 366.15, 'hydr. reaction pressure (bar)': 105.0, 'hydr. reaction yield': 1.0, 'hydr. reactor volume (m^3)': 1.0, 'number of hydr. 

### USER INPUT

In [3]:
# specify input parameter file
df_input_params_all = pd.read_excel('inputs/input_params_test.xlsx')

# specify output folder
output_folder = 'outputs/outputs ' + date.today().strftime("%Y-%m-%d") + ' test'

### RUN SYSTEMS ANALYSIS

In [4]:
# create output folder if not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [5]:
# fill NaN values in input parameter file
df_input_params_all.ffill(inplace = True)

df_input_params_all

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reaction pressure (bar),dehydr. reaction yield,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1.0,0.9999,0.073103,1,9.646712,3500.00,6,0.000000,0,1
1,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,1.0,0.9999,0.073103,1,9.646712,3500.00,6,0.000000,0,1
2,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1.0,0.9999,0.073103,1,9.646712,3500.00,6,0.000000,0,1
3,FA ideal,3,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,1.0,0.9999,0.073103,1,9.646712,3500.00,6,0.000000,0,1
4,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,2.5,0.9999,0.000000,0,41.850000,191.35,6,0.279301,0,1


In [6]:
# create output dataframe for all runs
df_output_all = pd.DataFrame()

# run systems analysis for each set of input parameters
for i in df_input_params_all.index:
    
    # create dictionary of input parameters for each run
    df_input_params = df_input_params_all.iloc[i]
    dict_input_params = df_input_params.to_dict()

    # run systems analysis
    df_output, _, _, _ = s2a_sys.calcs(
        dict_input_params, 
        save_csv = True,
        output_folder = output_folder
        )

    # add run number column and rearrange columns
    cols = df_output.columns.tolist()
    df_output['run #'] = df_input_params['run #']
    df_output = df_output[['run #'] + cols]

    # add output dataframe to dataframe for all runs
    df_output_all = pd.concat([df_output_all, df_output])

In [7]:
# save output dataframe for all runs as csv
df_output_all.to_csv(
    os.path.join(output_folder, 'output_all runs.csv'),         
    index = False
    )

df_output_all

,run #,pathway,process,location,function,equipment,variable group,variable name,unit,value
0,0,all,all,all,all,all,input parameter,scenario,-,baseline
1,0,all,all,all,all,all,input parameter,run #,-,0
2,0,all,all,all,all,all,input parameter,output dollar year,-,2022
3,0,all,all,all,all,all,input parameter,target station capacity,kg/day,1000
4,0,all,all,all,all,all,input parameter,station annual capacity factor,-,1.0
...,...,...,...,...,...,...,...,...,...,...
448,4,LOHC - MCH (ideal),reconditioning,refueling station,storage,liquid storage,capital cost,levelized capital cost,$/kg H2,0.020033
449,4,LOHC - MCH (ideal),reconditioning,refueling station,storage,cascade storage,capital cost,levelized capital cost,$/yr,1202355.235363
450,4,LOHC - MCH (ideal),reconditioning,refueling station,storage,cascade storage,capital cost,levelized capital cost,$/kg H2,0.329412
451,4,LOHC - MCH (ideal),reconditioning,refueling station,reaction,reactor,emissions,process emissions,kg CO2/kg H2,0.0


### PROCESS OUTPUT

In [8]:
# drop scenario and run number rows
df_output_all = df_output_all.loc[
    ~df_output_all['variable name'].isin(['scenario', 'run #'])
    ].reset_index(drop = True)

In [9]:
# filter for $/kg H2 cost results
df_usd_per_kg = df_output_all.loc[
    df_output_all['unit'] == '$/kg H2'
    ].reset_index(drop = True)

# filter for kg CO2/kg H2 emissions results
df_kgCO2_per_kg = df_output_all.loc[
    df_output_all['unit'] == 'kg CO2/kg H2'
    ].reset_index(drop = True)

#### OUTPUT: all input parameters

In [10]:
# write input parameters as csv to output folder
df_input_params_all.to_csv(
    os.path.join(output_folder, 'input params.csv'),         
    index = False
    )

#### OUTPUT: varied input parameters
including base and user-defined values 

In [11]:
# find column names of varied input parameters
varied_cols = []

for col in df_input_params_all.columns[2:]:
    if any(df_input_params_all[col] != df_input_params_all[col][0]):
        varied_cols.append(col)

# extract varied input parameters
df_input_params_varied = df_input_params_all[
    ['scenario', 'run #'] + varied_cols
    ]

# save dataframe as csv
df_input_params_varied.to_csv(
    os.path.join(output_folder, 'input params_varied.csv'),         
    index = False
    )

df_input_params_varied

,scenario,run #,station annual capacity factor,electricity emission factor (kg CO2/kWh),hydrogen purchase cost ($/kg),LOHC name,LOHC molar mass (kg/kmol),LOHC density (kg/m^3),stoic. ratio (mol H2/mol LOHC),stoic. ratio (mol CO2/mol LOHC),...,hydr. electrolyzer cell area (m^2/cell),hydr. electrolyzer voltage (V),dehydr. reaction pathway,dehydr. reaction temperature (K),dehydr. reaction pressure (bar),dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. plant energy (kWh/kg)
0,baseline,0,1.0,0.228,0.31,formic acid (ideal),46.0250,1220,1,1,...,5539.862206,1.27,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000
1,FA ideal - delivery only,1,1.0,0.000,0.00,formic acid (ideal),46.0250,1220,1,1,...,5539.862206,1.27,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000
2,FA ideal - 80 cap factor,2,0.8,0.000,0.31,formic acid (ideal),46.0250,1220,1,1,...,5539.862206,1.27,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000
3,FA ideal,3,1.0,0.000,0.31,formic acid (ideal),46.0250,1220,1,1,...,5539.862206,1.27,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000
4,MCH ideal,4,1.0,0.000,6.00,MCH (ideal),98.1861,770,3,0,...,5539.862206,1.08,"thermo, literature",633.15,2.5,0.000000,0,41.850000,191.35,0.279301


#### OUTPUT: refueling station costs for process modeling team

In [12]:
# set filters for refueling station costs
# including catalyst amount and reactor volume
pathway = ['LOHC - formic acid', 'all']
location = ['refueling station', 'all']
units = ['$/yr', '$/kg H2', '$/station', 'kg', 'm^3']
# equipment = [
#     'compressor',
#     'refrigerator',
#     'cryogenic pump',
#     'reactor pump',
#     'reactor',
#     'catalyst',
#     'PSA refrigerator',
#     'PSA'
#     ]

# filter results
df_stn = df_output_all.loc[
    df_output_all['pathway'].isin(pathway) & \
    df_output_all['location'].isin(location) & \
    df_output_all['unit'].isin(units)
    ].reset_index(drop = True)

# filter out hydrogenation inputs
df_stn = df_stn.loc[
    ~df_stn['variable name'].str.startswith('hydr.')
    ]

# save dataframe as csv
df_stn.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs.csv'
        ),         
    index = False
    )

# create column of output name
df_stn['name'] = \
    df_stn['equipment'] + ' ' + \
    df_stn['variable name'] + ' (' + \
    df_stn['unit'] + ')'

# drop unused columns
df_stn = df_stn[[
    'run #', 
    'name',
    'value',
    ]]

# df_stn

In [13]:
# pivot dataframe
df_stn_pivot = df_stn.pivot(
    columns = 'name', 
    index = 'run #', 
    values = 'value'
    ).reset_index()

# remove name of columns
df_stn_pivot.columns.name = None

# merge in varied input parameters
df_stn_pivot = df_input_params_varied.merge(
    right = df_stn_pivot, 
    on='run #', 
    how = 'right'
    )

# save dataframe as csv
df_stn_pivot.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs_pivot.csv'
        ),         
    index = False
    )

df_stn_pivot

,scenario,run #,station annual capacity factor,electricity emission factor (kg CO2/kWh),hydrogen purchase cost ($/kg),LOHC name,LOHC molar mass (kg/kmol),LOHC density (kg/m^3),stoic. ratio (mol H2/mol LOHC),stoic. ratio (mol CO2/mol LOHC),...,dehydr. reaction pathway,dehydr. reaction temperature (K),dehydr. reaction pressure (bar),dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. plant energy (kWh/kg),all dehydr. catalyst amount (kg),all dehydr. reactor volume (m^3)
0,baseline,0,1.0,0.228,0.31,formic acid (ideal),46.0250,1220,1,1,...,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000,9.646712,0.073103
1,FA ideal - delivery only,1,1.0,0.000,0.00,formic acid (ideal),46.0250,1220,1,1,...,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000,9.646712,0.073103
2,FA ideal - 80 cap factor,2,0.8,0.000,0.31,formic acid (ideal),46.0250,1220,1,1,...,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000,9.646712,0.073103
3,FA ideal,3,1.0,0.000,0.31,formic acid (ideal),46.0250,1220,1,1,...,"thermo, model",300.00,1.0,0.073103,1,9.646712,3500.00,0.000000,9.646712,0.073103
4,MCH ideal,4,1.0,0.000,6.00,MCH (ideal),98.1861,770,3,0,...,"thermo, literature",633.15,2.5,0.000000,0,41.850000,191.35,0.279301,41.85,0.0


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway

In [14]:
# calculate total $/kg H2 cost by pathway
df_usd_per_kg_path = df_usd_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_path.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_path = df_input_params_all.merge(
    right = df_usd_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_cost by pathway.csv'),         
    index = False
    )

df_usd_per_kg_path

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),36.657787
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,6.171953
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,7.272966
3,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),16.415
4,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,5.861953
5,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,6.962966
6,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),42.498225
7,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,7.343494
8,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,8.490938
9,FA ideal,3,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),36.657787


In [15]:
# calculate total kg CO2/kg H2 emissions by pathway
df_kgCO2_per_kg_path = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_path.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_path = df_input_params_all.merge(
    right = df_kgCO2_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_emissions by pathway.csv'),         
    index = False
    )

df_kgCO2_per_kg_path

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),9.923975
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,1.169879
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,2.629718
3,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),22.04466
4,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,0.325656
5,FA ideal - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,0.093998
6,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),0.214799
7,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,0.325656
8,FA ideal - 80 cap factor,2,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,0.093998
9,FA ideal,3,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),0.214799


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and process

In [16]:
# calculate total $/kg H2 cost by pathway and process
df_usd_per_kg_proc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc = df_input_params_all.merge(
    right = df_usd_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_cost by process.csv'),         
    index = False
    )

df_usd_per_kg_proc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),preconditioning,0.005758
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,29.284004
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),reconditioning,6.373528
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),transport,0.994497
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,preconditioning,2.48097
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,production,0.31
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,reconditioning,2.503494
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,transport,0.877489
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,preconditioning,4.787332
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,production,0.31


In [17]:
# calculate total kg CO2/kg H2 emissions by pathway and process
df_kgCO2_per_kg_proc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_emissions by process.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),preconditioning,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,7.699788
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),reconditioning,2.009389
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),transport,0.214799
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,preconditioning,0.533237
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,production,0.0
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,reconditioning,0.310986
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,transport,0.325656
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,preconditioning,2.423692
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,production,0.0


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and location

In [18]:
# calculate total $/kg H2 cost by pathway and location
df_usd_per_kg_loc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_loc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_loc = df_input_params_all.merge(
    right = df_usd_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_cost by location.csv'),         
    index = False
    )

df_usd_per_kg_loc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,location,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),refueling station,6.373528
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),terminal,19.148471
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),truck,11.135788
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,refueling station,2.503494
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,terminal,2.79097
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,truck,0.877489
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,liquefier,4.435231
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,refueling station,1.866226
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,terminal,0.662101
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,truck,0.309408


In [19]:
# calculate total kg CO2/kg H2 emissions by pathway and location
df_kgCO2_per_kg_loc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_loc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_loc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_emissions by location.csv'),         
    index = False
    )

df_kgCO2_per_kg_loc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,location,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),refueling station,2.009389
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),terminal,7.699788
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),truck,0.214799
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,refueling station,0.310986
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,terminal,0.533237
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,compressed hydrogen,truck,0.325656
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,liquefier,2.423572
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,refueling station,0.112027
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,terminal,0.000121
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,liquid hydrogen,truck,0.093998


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and function

In [20]:
# calculate total $/kg H2 cost by pathway and function
df_usd_per_kg_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_func = df_input_params_all.merge(
    right = df_usd_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_cost by function.csv'),         
    index = False
    )

df_usd_per_kg_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,function,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),CO2 recycling,10.141291
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),compression,2.604754
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),cooling,0.174649
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),pumping,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),purchase,0.310031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,pumping,1.242008
101,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,purchase,6.0
102,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,storage,0.930759
103,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,trucking,0.309408


In [21]:
# calculate total kg CO2/kg H2 emissions by pathway and function
df_kgCO2_per_kg_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_emissions by function.csv'),         
    index = False
    )

df_kgCO2_per_kg_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,function,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),CO2 recycling,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),compression,0.567446
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),cooling,0.079846
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),pumping,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),purchase,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,compressed hydrogen,trucking,0.325656
76,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,liquefaction,0.0
77,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,pumping,0.0
78,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,purchase,0.0


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway, process, and function

In [22]:
# calculate total $/kg H2 cost by pathway, process, and function
df_usd_per_kg_proc_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc_func = df_input_params_all.merge(
    right = df_usd_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_cost by process-function.csv'),         
    index = False
    )

df_usd_per_kg_proc_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,function,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),preconditioning,pumping,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),preconditioning,storage,0.005758
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,CO2 recycling,10.141291
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,compression,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,pumping,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,production,purchase,6.0
151,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,reconditioning,pumping,1.220412
152,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,reconditioning,storage,0.600254
153,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,reconditioning,vaporization,0.04556


In [23]:
# calculate total kg CO2/kg H2 emissions by pathway, process, and function
df_kgCO2_per_kg_proc_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_emissions by process-function.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. plant energy (kWh/kg),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,function,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),preconditioning,pumping,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,CO2 recycling,0.0
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,compression,0.0
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,purchase,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500.00,6,0.000000,0,1,LOHC - formic acid (ideal),production,reaction,7.699788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,preconditioning,liquefaction,0.0
101,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,preconditioning,pumping,0.0
102,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,production,purchase,0.0
103,MCH ideal,4,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,41.850000,191.35,6,0.279301,0,1,liquid hydrogen,reconditioning,pumping,0.0
